In [ ]:
# copy model from get_data.py
# replace list of cities by just one city.

In [ ]:
# error on line 81

# KeyError                                  Traceback (most recent call last)
# Cell In[54], line 1
# ----> 1 forecast.get_weather_forecast()

# Cell In[51], line 81, in WeatherForecast.get_weather_forecast(self)
#      74 # So we make the request to the weather API archive
#      75 weather_forecast_response= requests.get('https://api.open-meteo.com/v1/forecast',
#      76                     params = {'latitude': lat,
#      77                                 'longitude': lan,
#      78                                 'forecast_days' : self.days,
#      79                                 'hourly': weather_params,
#      80                                 'timezone': 'auto'}).json()
# ---> 81 weather_forecast_df = pd.DataFrame(weather_forecast_response['hourly'], columns = ['time'] + weather_params)
#      82 weather_forecast_df['time'] = pd.to_datetime(weather_forecast_df['time'], format='%Y-%m-%d')
#      83 weather_forecast_df = weather_forecast_df.set_index('time')

# KeyError: 'hourly'

In [6]:
import pandas as pd
import requests
from datetime import date
from dateutil.relativedelta import relativedelta
from geopy.geocoders import Nominatim

In [51]:
class WeatherForecast:
    def __init__(self, city:str, days:int):
        self.city = city
        self.days = days

    def get_city_lonlan(self):
        '''
        This function receives the name of one city and returns the lat and lon of that city
        in a dictionary
        '''

        # Create a geolocator object
        geolocator = Nominatim(user_agent="my_app")

        #save the coordinates of each city in self.city in a dictionary
        coordinates = {}
        
        # Get the location of the city
        location = geolocator.geocode(self.city)

        #check if the location exists
        if location:
            lat, lon = location.latitude, location.longitude # Extract the latitude and longitude
            coordinates[self.city] = [lat,lon]
        else:
            print(f"Could not retrieve coordinates for {self.city}")

        return coordinates

    def get_weather_forecast(self):

        '''
        This function receives 
            * the name of the city list
            * a number of days of weather forecast we want to work on
            
        This function returns a dataframe with the average of the weather data from these city list during those days of forecast
        '''

        # First we declare the weather parameters. Here we'll be taking all params supported by the API
        weather_params = ['temperature_2m','relativehumidity_2m','dewpoint_2m',
                      'apparent_temperature','pressure_msl','surface_pressure',
                      'precipitation','rain','snowfall','cloudcover',
                      'cloudcover_low','cloudcover_mid','cloudcover_high',
                      'shortwave_radiation','direct_radiation','direct_normal_irradiance',
                      'diffuse_radiation','windspeed_10m','windspeed_100m',
                      'winddirection_10m','winddirection_100m','windgusts_10m',
                      'et0_fao_evapotranspiration','weathercode','vapor_pressure_deficit',
                      'soil_temperature_0_to_7cm','soil_temperature_7_to_28cm',
                      'soil_temperature_28_to_100cm','soil_temperature_100_to_255cm',
                      'soil_moisture_0_to_7cm','soil_moisture_7_to_28cm',
                      'soil_moisture_28_to_100cm','soil_moisture_100_to_255cm']

        # Then we compute the dates used to get the weather forecast data
        
        # end_date : today + number of days of forecast (self.days)
        end_date = (date.today() + relativedelta(days=self.days)).strftime('%Y-%m-%d')
        
        #start_date : today at midnight
        start_date = (date.today()).strftime('%Y-%m-%d')

        #call the method to receive the coordinates from the self.city list
        coordinates = self.get_city_lonlan()
        
        #create an empty dataframe
        weather_forecast_df_full = pd.DataFrame(columns=weather_params)
        # cities = []
        
        #create a dataframe with weather params for the selected city and store it
        lat = coordinates[self.city][0]
        lan = coordinates[self.city][1]

    
        # So we make the request to the weather API archive
        weather_forecast_response= requests.get('https://api.open-meteo.com/v1/forecast',
                            params = {'latitude': lat,
                                        'longitude': lan,
                                        'forecast_days' : self.days,
                                        'hourly': weather_params,
                                        'timezone': 'auto'}).json()
        weather_forecast_df = pd.DataFrame(weather_forecast_response['hourly'], columns = ['time'] + weather_params)
        weather_forecast_df['time'] = pd.to_datetime(weather_forecast_df['time'], format='%Y-%m-%d')
        weather_forecast_df = weather_forecast_df.set_index('time')

        # Format float to 1 decimal, sum the 3 tables and return the average
        pd.options.display.float_format = "{:,.1f}".format

        return weather_forecast_df_full

In [52]:
forecast = WeatherForecast(city="Amiens", days=3)

In [53]:
forecast.get_city_lonlan()

{'Amiens': [49.8941708, 2.2956951]}

In [54]:
forecast.get_weather_forecast()

KeyError: 'hourly'